# Surprise homework

In [1]:
import io 
from collections import defaultdict
import pandas as pd

from surprise import Dataset
from surprise import SVD
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import accuracy
from surprise.model_selection import cross_validate

from surprise.model_selection import train_test_split


### Functions

In [2]:
# используйте полезные функции из FAQ
def precision_recall_at_k(predictions, k=5, threshold=3.52):
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    

    return pd.Series(precisions).mean(), pd.Series(recalls).mean()

def get_top_n(predictions, n=5):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


### Load data

In [3]:
data = Dataset.load_builtin('ml-100k')

### Define algorithms

In [4]:
# внимательно изучите документацию по метрикам и алгоритмам
algoNormal = NormalPredictor()
#Cos
sim_options = {'name': 'cosine',
               'user_based': True
               }
algoKnnCos = KNNBasic(sim_options=sim_options)

#MSD
sim_options = {'name': 'msd',
               'user_based': True
               }
algoKnnMSD = KNNBasic(sim_options=sim_options)

#Pearson
sim_options = {'name': 'pearson',
               'user_based': True
               }
algoKnnPearson = KNNBasic(sim_options=sim_options)

#SVD
algoSVD = SVD()

### Select algorithm

In [5]:
cross_validate(algoNormal, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5086  1.5161  1.5112  1.5183  1.5288  1.5166  0.0070  
Fit time          0.15    0.19    0.18    0.19    0.18    0.18    0.01    
Test time         0.21    0.20    0.16    0.16    0.20    0.18    0.02    


{'test_rmse': array([1.50855439, 1.51610107, 1.51115403, 1.51831248, 1.52877658]),
 'fit_time': (0.15012192726135254,
  0.18849658966064453,
  0.18051981925964355,
  0.18604707717895508,
  0.17552924156188965),
 'test_time': (0.20799684524536133,
  0.19747257232666016,
  0.15857315063476562,
  0.1591031551361084,
  0.19747209548950195)}

In [6]:
cross_validate(algoKnnCos, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0088  1.0103  1.0203  1.0144  1.0283  1.0164  0.0072  
Fit time          2.16    2.13    2.09    2.22    2.17    2.15    0.04    
Test time         4.96    4.70    4.82    5.25    4.71    4.89    0.20    


{'test_rmse': array([1.00878821, 1.01033848, 1.02034845, 1.01440192, 1.02829344]),
 'fit_time': (2.1565654277801514,
  2.131878614425659,
  2.092487335205078,
  2.2166366577148438,
  2.1708898544311523),
 'test_time': (4.955709934234619,
  4.70090126991272,
  4.819304943084717,
  5.2548441886901855,
  4.714485168457031)}

In [7]:
cross_validate(algoKnnMSD, data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9732  0.9889  0.9770  0.9820  0.9737  0.9790  0.0059  
Fit time          0.67    0.66    0.67    0.68    0.70    0.68    0.01    
Test time         4.61    4.68    4.73    4.78    4.68    4.70    0.06    


{'test_rmse': array([0.97319146, 0.98886972, 0.97703924, 0.98199357, 0.97372519]),
 'fit_time': (0.6712090969085693,
  0.6616823673248291,
  0.6731977462768555,
  0.6775913238525391,
  0.7041163444519043),
 'test_time': (4.612980842590332,
  4.684175968170166,
  4.729371070861816,
  4.781862020492554,
  4.684913873672485)}

In [8]:
cross_validate(algoKnnPearson, data, measures=['RMSE'], cv=5, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0047  1.0172  1.0085  1.0155  1.0152  1.0122  0.0048  
Fit time          2.58    2.74    2.64    3.03    2.67    2.73    0.16    
Test time         4.58    4.58    4.72    4.73    4.75    4.67    0.08    


{'test_rmse': array([1.00469795, 1.0172177 , 1.0085249 , 1.01550061, 1.01523867]),
 'fit_time': (2.5816564559936523,
  2.7430384159088135,
  2.6377556324005127,
  3.0279107093811035,
  2.6669535636901855),
 'test_time': (4.578812837600708,
  4.576437711715698,
  4.722501993179321,
  4.7317750453948975,
  4.754007577896118)}

In [9]:
cross_validate(algoSVD, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9356  0.9347  0.9305  0.9346  0.9399  0.9351  0.0030  
Fit time          5.81    5.95    5.88    5.84    5.93    5.88    0.05    
Test time         0.20    0.25    0.24    0.18    0.25    0.23    0.03    


{'test_rmse': array([0.93563401, 0.93471796, 0.93045374, 0.93456197, 0.9399344 ]),
 'fit_time': (5.809868574142456,
  5.953454494476318,
  5.877994775772095,
  5.8378005027771,
  5.927286148071289),
 'test_time': (0.20242714881896973,
  0.25232553482055664,
  0.24434328079223633,
  0.17607331275939941,
  0.2533237934112549)}

### Calculate precision@k and recall@k

In [10]:
trainset, testset = train_test_split(data, test_size=.25)
algoSVD.fit(trainset)
predictions = algoSVD.test(testset)

print(precision_recall_at_k(predictions))

(0.7313184870979145, 0.36428043116661235)


### Predict

In [11]:
variant = '12'
user_prediction = get_top_n(predictions)[variant]


In [12]:
item_path = '/Users/Maxim/.surprise_data/ml-100k/ml-100k/u.item'
item_df = pd.read_csv(item_path, sep='|',encoding='ISO-8859-1', header = None, index_col = 0)
item_df

,1,2,3,4,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,22,23
0,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [13]:
print("User " + variant)
for item in user_prediction:
    film_name = item_df.loc[int(item[0])][1]
    film_date = item_df.loc[int(item[0])][2]
    print(item[0] + " ('" + film_name + "', '" + film_date + "') " + str(round(item[1],3)))
    

User 12
50 ('Star Wars (1977)', '01-Jan-1977') 4.863
97 ('Dances with Wolves (1990)', '01-Jan-1990') 4.499
82 ('Jurassic Park (1993)', '01-Jan-1993') 4.391
133 ('Gone with the Wind (1939)', '01-Jan-1939') 4.377
735 ('Philadelphia (1993)', '01-Jan-1993') 4.355
